In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys

sys.path.append("../")
from C4theo_asymptotiqueValue_Calculus import C4_short, C4_long
from Physic_function_problem import *
import Functions_cythonised

from scipy.integrate import quad
from scipy.io import loadmat, savemat
from scipy.optimize import curve_fit,minimize
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from numpy import trapz
import matplotlib.gridspec as gridspec
from matplotlib.collections import LineCollection
from mpltools import annotation
import glob

def cm2inch(value):
    return value / 2.54

from tqdm import tqdm


from matplotlib import rc
import seaborn as sns
custom_params = {
    "xtick.direction": "in",
    "ytick.direction": "in",
    "lines.markeredgecolor": "k",
    "lines.markeredgewidth": 0.3,
    "figure.dpi": 200,
    "text.usetex": True,
    "font.family": "serif",
    # "font.size":8,
    # 'axes.titlesize':10,
    # 'axes.labelsize':10,
}
# sns.set_context("notebook", rc={"lines.linewidth": 3})


sns.set_theme(context = "paper", style="ticks", rc=custom_params)

BlueUB = (0, 157/255, 224/255)
BrownUB = (68/255, 58/255, 49/255)

### Simulations/Experimental parameter

In [ ]:
a = 1.519e-6 #1.12e-6/2
H = 40e-6
lD = 88.0e-9 #Répultion mur
kBT=1.38e-23*300
lB = 526e-9 #gravité
B = 5.0
eta=0.001

zz = np.linspace(-H+H*1e-4, H-H*1e-4, 1000)

H = 40e-6
lD = 88.0e-9 #Répultion mur
kBT=1.38e-23*300
lB = 526e-9 #gravité
B = 5.0
eta=0.001

dt = 1e-2
Nt = 100000
Nt_sub = 1
tt = np.linspace(dt, dt*Nt, Nt)

### Some functions to analyse

In [ ]:
def logarithmic_hist(data: np.ndarray, begin: float, stop: float, num: int = 50, base: int = 10) -> (np.ndarray, np.ndarray, np.ndarray):
        """Function to compute a pdf using  logspaced bins"""
        if begin == 0:
            beg = stop / num
            bins = np.logspace(
                np.log(beg) / np.log(base),
                np.log(stop) / np.log(base),
                num - 1,
                base=base,
            )
            widths = bins[1:] - bins[:-1]
            bins = np.cumsum(widths[::-1])
            bins = np.concatenate(([0], bins))
            widths = bins[1:] - bins[:-1]

        else:
            bins = np.logspace(
                np.log(begin) / np.log(base),
                np.log(stop) / np.log(base),
                num,
                base=base,
            )
            widths = bins[1:] - bins[:-1]

        hist, bins = np.histogram(data, bins=bins, density=True)
        # normalize by bin width
        bins_center = (bins[1:] + bins[:-1]) / 2

        return bins_center, widths, hist

### Theoritical values

In [ ]:
mean_Dx = np.trapz(P_eq(zz, B, lD, lB, H)/np.trapz(P_eq(zz, B, lD, lB, H), zz)*D_x(zz, a, eta, H), zz)
mean_Dz = np.trapz(P_eq(zz, B, lD, lB, H)/np.trapz(P_eq(zz, B, lD, lB, H), zz)*D_z(zz, a, eta, H), zz)

In [ ]:
N_tau = 1


##THEORIE

dXX = np.linspace(-0.5, +0.5, 1000)
PDF_theory_x = P_Di_short_time(dXX*1e-6, dt*N_tau, D_x, a, eta, B, lD, lB, H)

dZZ = np.linspace(-0.8, +0.8, 1000)
PDF_theory_z = P_Di_short_time(dZZ*1e-6, dt*N_tau, D_z, a, eta, B, lD, lB, H)

In [ ]:
dZZ_long = np.linspace(-2*(H), 2*(H), 10000)
PDF_theory_z_long = Pdeltaz_long(dZZ_long, B, lD, lB, H)


plateau = np.trapz(dZZ_long**2 * PDF_theory_z_long, dZZ_long)

plateauC4z = np.trapz(dZZ_long**4 * PDF_theory_z_long, dZZ_long)
plateauC4z = np.trapz(dZZ_long**4 * PDF_theory_z_long , dZZ_long) - 3 * np.trapz(dZZ_long**2 * PDF_theory_z_long , dZZ_long) ** 2 

In [ ]:
np.sqrt(plateau)*1e6

In [ ]:
D4_long_theo = 1.2691e-27 #1.5074e-27
C4_long_theo = 1.0350e-26 #1.4039e-26
A4_theo_x = C4_court(D=D_para, Peq=P_eq, kBT=kBT, B=B, lD=lD, lB=lB, H=H, a=a, eta=eta, dx=1e-8)
A4_theo_z = C4_court(D=D_perp, Peq=P_eq, kBT=kBT, B=B, lD=lD, lB=lB, H=H, a=a, eta=eta, dx=1e-8)

t_short = np.linspace(1e-4, 50, 1000)
t_long = np.linspace(20, 1e5, 1000)

# Datas

In [ ]:
# X datas
Datas_parquet = "../C4_Datas/convergence_7_50_sigma__lD_8p800000e-08_lB_5p260000e-07_B_5p000000e+00_H_4p000000e-05_a_1p519000e-06_dt_1p000000e-02_Nt_1000000_Ntsub_1_/"

dt = float(Datas_parquet.replace("p", ".").split("_")[-6])

# #Récupération des datas
# pd.read_parquet(Datas_parquet+'/1', engine='pyarrow')
files=glob.glob(Datas_parquet+"/*")

# # Les mettres dans l'ordre
Ntime = []
for i in files:
    Ntime.append(int(i.split("/")[-1]))
I=np.argsort(Ntime)
files = [files[i] for i in I]
Ntime = np.sort(Ntime)

np.shape(files), Ntime

In [ ]:
# Z datas
datasZnpy = '../C4_Datas/PDFz_Ns100.npy'
SumHistZ_Ns100 = np.load(datasZnpy) #PDF(\tau, \Delta Z) sum over 100 trajectories
Ntau, bins = np.shape(SumHistZ_Ns100)
Ns = 100

list_Ntau = np.logspace(np.log(1)/np.log(10), np.log(1000001)/np.log(10),120).astype(int)
list_Ntau = np.unique(list_Ntau)
tau_z = np.array(list_Ntau) * dt #list of lag time to cumpue C4


listdZ = np.zeros([Ntau,bins])
D_0 = kBT/(6*np.pi*eta*a)
for n, i in enumerate(list_Ntau):
    tau = dt*i
    if tau<50:
        Sigma = np.sqrt(2*D_0*tau)
        listdZ[n,:] = np.linspace(-6*Sigma,+6*Sigma, bins)
    else:
        listdZ[n,:] = np.linspace(-15e-6, 15e-6, bins)

In [ ]:
# # Z datas (À SUPPRIMER)
# Datas_parquet_z = "../C4_Datas/z_convergence_7_50_sigma__lD_8p800000e-08_lB_5p260000e-07_B_5p000000e+00_H_4p000000e-05_a_1p519000e-06_dt_1p000000e-02_Nt_1000000_Ntsub_1_/"

# #Take datas
# files_z=glob.glob(Datas_parquet_z+"/*")
# #Ordering datas
# Ntime_z = []
# for i in files_z:
#     Ntime_z.append(int(i.split("/")[-1]))
# I=np.argsort(Ntime_z)
# files_z = [files_z[i] for i in I]
# Ntime_z = np.sort(Ntime_z)
# np.shape(Ntime_z), Ntime_z

In [ ]:
# del Datas_parquet, Datas_parquet_z

## MSD à partir des datas du papiers C4

In [ ]:
tau_theo = np.logspace(-3,4, 100)

In [ ]:
# MSDx
dt = np.array(float(Datas_parquet.replace("p", ".").split("_")[-6]))
Nt = np.array(float(Datas_parquet.replace("p", ".").split("_")[-4]))
tau_x = Ntime*dt

tau_theo = np.linspace(1e-4, 1e4, 100)

In [ ]:
# Moments = {}
# Moments["tau_x"] = tau_x
Moments["tau_z"] = tau_z

In [ ]:
m = 2 #Choose order of moment
axe = "x" #Choose axis
key = "<"+str(axe)+str(m)+">"
Moments[key] = np.zeros(len(files))

for n, i in enumerate(tqdm(files)): #loop for all time \tau
    datas = pd.read_parquet(i)
    dx = np.array(datas.columns).astype(float)
    #Average over Ns distributions
    datas = datas.to_numpy()
    datas = np.sum(datas, axis=0)
    datas = datas / np.trapz(datas, dx)
    #Comupte <X^m>(\tau)
    Moments[key][n] = (np.trapz(dx**m *datas, dx))


In [ ]:
m = 2 #Choose order of moment
axe = "z" #Choose axis
key = "<"+str(axe)+str(m)+">"
Moments[key] = np.zeros(len(list_Ntau))
for n, i in enumerate(list_Ntau[:-1]):
    dZ = listdZ[n,:]
    tau = i*dt
    PDFs_z = SumHistZ_Ns100[n,:] / np.trapz(SumHistZ_Ns100[n,:], dZ) #Normalised
    Moments[key][n] = (np.trapz(dZ**m *PDFs_z, dZ))



In [ ]:
plt.figure(figsize=(3.375, 2*3.375 / 1.68), tight_layout=True)

ax1 = plt.subplot(211)
ax1.loglog(Moments["tau_x"], Moments["<x2>"], "o", color=BlueUB)
ax1.plot(tau_theo, 2*mean_Dx*tau_theo, "--", color="k")
ax1.loglog(Moments["tau_z"], Moments["<z2>"] , "d", color="tab:green")
ax1.plot(tau_theo, 2*mean_Dz*tau_theo, "-", color="k") 
ax1.plot(np.linspace(10, 10000, 100), plateau*np.ones(100), ":", color="k")
ax1.set(
    # xlabel = r"$\tau ~\mathrm{(s)}$",
    ylabel = r"$MSD~\mathrm{(\mu m)}^2$",
    xlim=(dt/2, dt*Nt/10),
    # ylim=(1e-16*1e12, 2e-10*1e12),
)

In [ ]:
# from scipy.io import savemat
# Moments['PARAMETRES:_a_eta_H_lD_lB_B_Nt_Ntsub_Ns'] = np.array([a, eta, H, lD, lB, B, Nt, Nt_sub, Ns])

# savemat("../Large_Gap/SIMU_Moments_Nt_1e6_dt_1e-2.mat", Moments)

# Cumulant

In [ ]:
Cumulant4 = {}
Cumulant4["<x4>"] = np.zeros(len(files))
Cumulant4["<z4>"] = np.zeros(len(list_Ntau))

In [ ]:
for n, i in enumerate(tqdm(files)): #loop for all time \tau
    datas = pd.read_parquet(i)
    dx = np.array(datas.columns).astype(float)
    #Average over Ns distributions
    datas = datas.to_numpy()
    datas = np.sum(datas, axis=0)
    datas = datas / np.trapz(datas, dx)
    #Comupte cumualnt
    Cumulant4["<x4>"][n] = (np.trapz(dx**4 *datas, dx)) - 3 * (np.trapz(dx**2 *datas, dx))**2

In [ ]:
for n, i in enumerate(list_Ntau[:-1]):
    dZ = listdZ[n,:]
    tau = i*dt
    PDFs_z = SumHistZ_Ns100[n,:] / np.trapz(SumHistZ_Ns100[n,:], dZ) #Normalised
    Cumulant4["<z4>"][n] = (np.trapz(dZ**4 *PDFs_z, dZ)) - 3 * (np.trapz(dZ**2 *PDFs_z, dZ))**2



In [ ]:
plt.figure(figsize=(3.375, 3.375 / 1.68), tight_layout=True)


plt.loglog(tau_x, Cumulant4["<x4>"], "o")
plt.loglog(tau_z, Cumulant4["<z4>"], "d")


# Methode $2$ to compute

In [ ]:
# # Calcul C4 avec MÉTHODE_2 PDF -> C4 -> <C4>
# Cumulant4["<x4> by avering over Ns <x4>"] = np.zeros(len(files))
# for n, i in enumerate(tqdm(files)): #loop for all time \tau
#     datas = pd.read_parquet(i)
#     dx = np.array(datas.columns).astype(float)
#     #Average over Ns distributions
#     datas = datas.to_numpy()
#     tampon = []
#     for j in (range(len(datas[:,0]))):
#         if sum(datas[j, :]) != 0:
#             #Calcul of cumulant on jth PDF
#             PDF = datas[j,:] / np.trapz(datas[j,:], dx)
#             tampon.append( np.trapz(dx**4 *PDF, dx) - 3* (np.trapz(dx**2 * PDF, dx))**2 )
    
#     #Mean over Ns cumulants
#     Cumulant4["<x4> by avering over Ns <x4>"][n] = np.mean(tampon)

    
# del tampon
# del PDF

In [ ]:
# from scipy.io import loadmat

# Cumulant_ = loadmat("../Large_Gap/SIMU_Cumulants_Nt_1e6_dt_1e-2.mat", squeeze_me=True)
# del Cumulant_

In [ ]:
# Cumulant4["<x4> by avering over Ns <x4>"] = Cumulant_['<x4> by avering over Ns <x4>']

In [ ]:
plt.figure(figsize=(3.375, 3.375 / 1.68), tight_layout=True)

plt.loglog(tau_x, Cumulant4["<x4>"], "o")
plt.loglog(tau_x, Cumulant4["<x4> by avering over Ns <x4>"], "^")


In [ ]:
from scipy.io import savemat
Cumulant4['PARAMETRES:_a_eta_H_lD_lB_B_Nt_Ntsub'] = np.array([a, eta, H, lD, lB, B, Nt, Nt_sub])
Cumulant4["tau_x"] = tau_x
Cumulant4["tau_z"] = tau_z
Cumulant4["Origine_datas_(X,Z)"] = Datas_parquet, datasZnpy
savemat("../Large_Gap/SIMU_Cumulants_Nt_1e6_dt_1e-2.mat", Cumulant4)

# CALCUL Kurtosis

In [ ]:
Kurtosis = {}
Kurtosis["x"] = np.zeros(len(files))
for n,i in enumerate(tqdm(files)):
    datas = pd.read_parquet(i)
    dx = np.array(datas.columns).astype(float)
    
    #Moyenne des PDFs
    datas = datas.to_numpy()
    datas = np.sum(datas, axis=0)
    datas = datas / np.trapz(datas, dx)
    
    #Kurtosis
    Kurtosis["x"][n] = (np.trapz(dx**4 *datas, dx)/(np.trapz(dx**2 * datas, dx)**2) - 3)



In [ ]:
Kurtosis['PARAMETRES:_a_eta_H_lD_lB_B_Nt_Ntsub'] = np.array([a, eta, H, lD, lB, B, Nt, Nt_sub])
Kurtosis["tau_x"] = tau_x
savemat("../C4_Datas/SIMU_Kurtosis_Nt_1e6_dt_1e-2.mat", Kurtosis)

# PDFs at $\tau_0$, $\tau_\mathrm{mid}$, $\tau_\mathrm{long}$ & $\tau_\infty$

In [ ]:
n_court = 0
n_mid = -17
n_long = -11
n_inf = -8
n_list = np.array([n_court, n_mid, n_long, n_inf], dtype=int)

In [ ]:
n_court_z = 0
n_mid_z = -53
n_long_z = -41
n_inf_z = -35
n_list_z = np.array([n_court_z, n_mid_z, n_long_z, n_inf_z], dtype=int)

In [ ]:
# Selection \tau for X datas
FilesX_selec = []
for n in n_list:
    FilesX_selec.append(files[n])

# Take datas
datasX_selec = {}
for i in FilesX_selec:
    name = i.split("/")[-1]
    datasX_selec[name] = pd.read_parquet(i)

Ntau_x_selec = list(datasX_selec.keys())

# Removings 0 lines
for i in Ntau_x_selec:
    datasX_selec[i] = datasX_selec[i].loc[datasX_selec[i].sum(axis = 1) != 0]


In [ ]:
Hist = {}
Hist['PARAMETRES:_a_eta_H_lD_lB_B_Nt_Ntsub'] = np.array([a, eta, H, lD, lB, B, Nt, Nt_sub])
Hist["tau_x_selec"] = np.array(list(map(float, Ntau_x_selec)) ) * dt
Hist["Datas_origines [x,z]"] = Datas_parquet, datasZnpy

for n,i in enumerate(Ntau_x_selec):#FOR X
    key_x = "dx_"+str(n)+ ", Sum_Hist_x"+str(n)
    Hist[key_x] = [np.array(datasX_selec[i].columns).astype(float), datasX_selec[i].to_numpy()]
    Hist[key_x][1] = np.sum(Hist[key_x][1], axis=0) #Sum to concerve counts

In [ ]:
for n,i in enumerate(n_list_z): #FOR Z
    key_z = "dz_"+str(n)+ ", Sum_Hist_z"+str(n)
    Hist[key_z] = np.zeros([bins,bins])
    Hist[key_z] = listdZ[i], SumHistZ_Ns100[i]

Hist["tau_z_selec"] = list_Ntau[n_list_z]*dt

In [ ]:
dz_0, SumHist_z0 = Hist['dz_0, Sum_Hist_z0']
dz_1, SumHist_z1 = Hist['dz_1, Sum_Hist_z1']
dz_2, SumHist_z2 = Hist['dz_2, Sum_Hist_z2']
dz_3, SumHist_z3 = Hist['dz_3, Sum_Hist_z3']

In [ ]:
dxCOURT = np.linspace(-0.4e-6, 0.4e-6, 1000)
dxLONG = np.linspace(-60e-6, 60e-6, 1000)
dzCOURT = np.linspace(-0.4e-6, 0.4e-6, 1000)
dzLONG = np.linspace(-2*H, 2*H, 1000)

dtsX = Hist["tau_x_selec"] 
dtsZ = Hist["tau_z_selec"] 

## Gaussian theories
Gauss_x0 = gauss(dxCOURT, 0, np.sqrt(2*mean_Dx*dtsX[0]))
Gauss_z0 = gauss(dzCOURT, 0, np.sqrt(2*mean_Dz*dtsZ[0]))
Gauss_x2 = gauss(dxLONG, 0, np.sqrt(2*mean_Dx*dtsX[2]))


In [ ]:
# Theory along Z
PDF_theory_z_short = P_Di_short_time(dzCOURT, dtsZ[0], D_z, a, eta, B, lD, lB, H)
PDF_theory_z_long = Pdeltaz_long(dzLONG, B, lD, lB, H)

In [ ]:
savemat("SIMU-SumHist_Ns-x_2097378_Ns-z_100_Nt_1e6_dt_1e-2_For4TimesTau.mat", Hist)

In [ ]:
plt.figure(figsize=(1.8 * 3.375, 1.4 * 3.375 / 1.68), tight_layout=True)
taille_marque = 6
taille_ligne = 2


#####################
ax2 = plt.subplot(121)#Short Z
ax2.plot(dzCOURT*1e6, Gauss_z0/np.trapz(Gauss_z0,dzCOURT*1e6), "--", color="tab:olive", linewidth=taille_ligne)
ax2.semilogy(dz_0*1e6,SumHist_z0/np.trapz(SumHist_z0,dz_0*1e6), "d", color="tab:green", markersize=taille_marque)
ax2.semilogy(dzCOURT*1e6, PDF_theory_z_short/np.trapz(PDF_theory_z_short, dzCOURT*1e6), "k-", linewidth=1.5)
ax2.set(
    ylabel = r"$P(\Delta Z, \tau)~(\mathrm{\mu m})^{-1}$",
    xlabel = r"$\Delta Z~\mathrm{(\mu m)}$", 
    xlim = (-0.3, 0.3),
    ylim = (1e-10, 5e1),
)
ax2.text(0.08, 0.9, r"(a)", transform=ax2.transAxes)
ax2.text(0.7, 0.9, r"$\tau_0= $ "+ str(dtsZ[0]) + " s", fontsize=9, transform=ax2.transAxes)
locmaj = mpl.ticker.LogLocator(base=10.0, subs=(1.0, ), numticks=100)
# locmin = mpl.ticker.LogLocator(base=10.0, subs=np.arange(2, 10) * .1, numticks=100)
ax2.yaxis.set_major_locator(locmaj)
ax2.yaxis.set_minor_locator(locmin)
ax2.yaxis.set_minor_formatter(mpl.ticker.NullFormatter())
ax2_ticks = ax2.get_yticklabels()
for i in ax2_ticks:
    i.set_visible(False)
for n, i in enumerate(ax2_ticks):
    if n%2==0:i.set_visible(True)

#####################
ax4 = plt.subplot(122)#Long Z
ax4.semilogy(dz_2*1e6,SumHist_z2/np.trapz(SumHist_z2,dz_2*1e6), "d", color="tab:green", markersize=taille_marque)
ax4.semilogy(dzLONG*1e6, PDF_theory_z_long/np.trapz(PDF_theory_z_long, dzLONG*1e6), "k-", linewidth=1.5)
ax4.set(
    xlabel = r"$\Delta Z~\mathrm{(\mu m)}$", 
    xlim = (-12,12),
    ylim = (1e-9,1e0),
)
ax4.text(0.08, 0.9, r"(b)", transform=ax4.transAxes)
ax4.text(0.7, 0.9, r"$\tau_\infty = $ "+ str(dtsZ[2]) + " s", fontsize=9, transform=ax4.transAxes)
locmaj = mpl.ticker.LogLocator(base=10.0, subs=(1.0, ), numticks=100)
# locmin = mpl.ticker.LogLocator(base=10.0, subs=np.arange(2, 10) * .1, numticks=100)
ax4.yaxis.set_major_locator(locmaj)
ax4.yaxis.set_minor_locator(locmin)
ax4.yaxis.set_minor_formatter(mpl.ticker.NullFormatter())
ax4_ticks = ax4.get_yticklabels()
for i in ax4_ticks:
    i.set_visible(False)
for n, i in enumerate(ax4_ticks):
    if n%2==0:i.set_visible(True)


# -----------------

# Cumulant 4

In [ ]:
dt = np.array(float(Datas_parquet_z.replace("p", ".").split("_")[-6]))
tau_x = Ntime*dt
tau_z= Ntime_z*dt


In [ ]:
data_x0 = pd.read_parquet(files[0])
data_z0 = pd.read_parquet(files_z[0])

np.shape(data_x0), np.shape(data_x0) 



# CALCUL C4_x

In [ ]:
# # CALCUL C4 ---------- <PDF> -> C4

C4_x = []
for i in tqdm(files):
    datas = pd.read_parquet(i)
    dx = np.array(datas.columns).astype(float)
    
    #Moyenne des PDFs
    datas = datas.to_numpy()
    datas = np.sum(datas, axis=0)
    datas = datas / np.trapz(datas, dx)
    
    #Calcul du C4_x
    C4_x.append(np.trapz(dx**4 *datas, dx) - 3* (np.trapz(dx**2 * datas, dx))**2)



# CALCUL C4_z

In [ ]:
# CALCUL C4_Z ----------- <PDF> -> C4

C4_z = []
for i in tqdm(files_z):
    datas_z = pd.read_parquet(i)
    dz = np.array(datas_z.columns).astype(float)
    
    #Moyenne des PDFs
    datas_z = datas_z.to_numpy()
    datas_z = np.sum(datas_z, axis=0)
    datas_z = datas_z / np.trapz(datas_z, dz)
    
    #Calcul du C4_x
    C4_z.append(np.trapz(dz**4 *datas_z, dz) - 3* (np.trapz(dz**2 * datas_z, dz))**2)



In [ ]:
# from scipy.io import savemat
# C4s = {}
# C4s['PARAMETRES:_a_eta_H_lD_lB_B_Ntsub'] = np.array([a, eta, H, lD, lB, B, Nt_sub])
# C4s["tau_x, C4_x"] =tau_x, C4_x
# C4s["tau_z, C4_z"] =tau_z, C4_z
# savemat("SIMU-C4s_Nt_1e6_dt_1e-2.mat", C4s)

In [ ]:
C4s =  loadmat("SIMU-C4s_Nt_1e6_dt_1e-2.mat", squeeze_me=True)

tau_x, C4_x = C4s["tau_x, C4_x"] 
tau_z, C4_z = C4s["tau_z, C4_z"] 


### - CALCUL C4_x méthode 2

In [ ]:
# Calcul C4 avec MÉTHODE_2 PDF -> C4 -> <C4>

# C4_xM2 = []

# for i in tqdm(files):
#     datas = pd.read_parquet(i)
#     dx = np.array(datas.columns).astype(float)
#     tampon = []
#     #PDFs
#     datas = datas.to_numpy()
#     for j in (range(len(datas[:,0]))):
#         if sum(datas[j, :]) != 0:
#             #Calcul des C4_x
#             PDF = datas[j,:] / np.trapz(datas[j,:], dx)
#             tampon.append( np.trapz(dx**4 *PDF, dx) - 3* (np.trapz(dx**2 * PDF, dx))**2 )
    
#     # Moyenne des C4s
#     C4_xM2.append(np.mean(tampon))

# del tampon
# del PDF


# from scipy.io import savemat

# C4_from_methode2 = {}
# C4_from_methode2["time (s)"] = tau_x
# C4_from_methode2["C4 (m^4)"] = C4_xM2

# savemat("C4_Datas/C4_ComputeWithMethode2_MeanOfC4.mat", C4_from_methode2)

M2_c4 = loadmat("../C4_Datas/C4_ComputeWithMethode2_MeanOfC4.mat", squeeze_me=True)
# np.shape(M2_c4["C4 (m^4)"])

In [ ]:
## COMPAISON DES DEUX MÉTHODES DE CALCUL DE C4

fig = plt.figure(figsize = (4, 4/1.68),  tight_layout=True)

ax1 = plt.subplot(111)

ax1.loglog(tau_x, C4_x, "o", color="tab:blue", label=r"$\int \Delta X^4 P(\Delta X) \mathrm{d}\Delta X - 3 \left( \int \Delta X^2 P(\Delta X) \mathrm{d}\Delta X \right)^2 $")
# ax1.loglog(M2_c4["time (s)"], M2_c4["C4 (m^4)"], "o", color="tab:pink", label=r"$\langle \Delta X^4 \rangle - 3 \langle \Delta X^2 \rangle^2$")
ax1.plot(tau_x[n_court], C4_x[n_court], "k*")
ax1.plot(tau_x[n_mid], C4_x[n_mid], "k*")
ax1.plot(tau_x[n_long], C4_x[n_long], "k*")
ax1.plot(t_short, A4_theo_x*t_short**2, "k-" )
ax1.plot(t_long, 24*D4_long_theo*t_long-24*C4_long_theo, "k--" )


plt.legend()

In [ ]:
# # CALCUL KURTOSIS Z

# Kurtosis_z = []
# for i in tqdm(files_z):
#     datas_z = pd.read_parquet(i)
#     dz = np.array(datas_z.columns).astype(float)
    
#     #Moyenne des PDFs
#     datas_z = datas_z.to_numpy()
#     datas_z = np.sum(datas_z, axis=0)
#     datas_z = datas_z / np.trapz(datas_z, dz)
    
#     #Calcul du C4_x_
#     Kurtosis_z.append(np.trapz(dz**4 *datas_z, dz)/(np.trapz(dz**2 * datas_z, dz)**2) - 3)



In [ ]:
# from scipy.io import savemat
# KURTOSIS = {}
# KURTOSIS['PARAMETRES:_a_eta_H_lD_lB_B_Ntsub'] = np.array([a, eta, H, lD, lB, B, Nt_sub])
# KURTOSIS["tau_x, Kurtosis_x"] =tau_x, Kurtosis_x
# KURTOSIS["tau_z, Kurtosis_z"] =tau_z, Kurtosis_z
# savemat("SIMU-KURTOSIS_Nt_1e6_dt_1e-2.mat", KURTOSIS)

In [ ]:
KURTOSIS =  loadmat("SIMU-KURTOSIS_Nt_1e6_dt_1e-2.mat", squeeze_me=True)

tau_x, Kurtosis_x = KURTOSIS["tau_x, Kurtosis_x"] 
tau_z, Kurtosis_z = KURTOSIS["tau_z, Kurtosis_z"] 


# DATA PDFs $\tau_1$,  $\tau_2$,  $\tau_3$, 

In [ ]:
filess = [files[n_court], files[n_mid], files[n_long], files[n_inf]]
filess_z = [files_z[n_court], files_z[n_mid], files_z[n_long]]

In [ ]:
tau_x[n_court], tau_x[n_mid], tau_x[n_long], tau_x[n_inf]

In [ ]:
datas = {}
for i in filess:
    name = i.split("/")[-1]
    datas[name] = pd.read_parquet(i)


    
datas_z = {}
for i in filess_z:
    name = i.split("/")[-1]
    datas_z[name] = pd.read_parquet(i)

In [ ]:
len(datas)

In [ ]:
times = list(datas.keys())

times_z = list(datas_z.keys())
times, times_z

In [ ]:
# removings 0 lines

for i in times_z:
    datas[i] = datas[i].loc[datas[i].sum(axis = 1) != 0]
    datas_z[i] = datas_z[i].loc[datas_z[i].sum(axis = 1) != 0]

In [ ]:
datas['19306'] = datas['19306'].loc[datas['19306'].sum(axis = 1) != 0]

In [ ]:
PDFs = {}
PDFs['PARAMETRES:_a_eta_H_lD_lB_B_Ntsub'] = np.array([a, eta, H, lD, lB, B, Nt_sub])
PDFs["dts"] = np.array(list(map(float, times)) ) * dt
PDFs["Readme"] = ["Only along x for last dts[3]"]
PDFs["Datas_origines [x,z]"] = [files[0].split("/")[-2], files_z[0].split("/")[-2]]

for n,i in enumerate(times):
    key = "dx_"+str(n)+ ", PDF_x"+str(n) 
    PDFs[key] = [np.array(datas[i].columns).astype(float), datas[i].to_numpy()]
    
    if i != '19306':
        key_z = "dz_"+str(n)+ ", PDF_z"+str(n) 
        PDFs[key_z] = [np.array(datas_z[i].columns).astype(float), datas_z[i].to_numpy()]


In [ ]:
PDFs[ "dts"]

In [ ]:
from scipy.io import savemat    
savemat("../C4_Datas/PLOT_PapierSimuC4/SIMU-PDFs_Nt_1e6_dt_1e-2_Version2_0p01-23p29-95p4-193p06.mat", PDFs)

In [ ]:
dxLONG = np.linspace(-60e-6, 60e-6, 1000)
dxMID = np.linspace(-15e-6, 15e-6, 1000)
dxCOURT = np.linspace(-0.4e-6, 0.4e-6, 1000)

dzCOURT = np.linspace(-0.4e-6, 0.4e-6, 1000)
dzLONG = np.linspace(-2*H, 2*H, 1000)

t_short = np.linspace(1e-4, 50, 1000)
t_long = np.linspace(20, 1e5, 1000)

In [ ]:
dx_0, data_0 = PDFs['dx_0, PDF_x0']
dx_1, data_1 = PDFs['dx_1, PDF_x1']
dx_2, data_2 = PDFs['dx_2, PDF_x2']

dz_0, data_z0 = PDFs['dz_0, PDF_z0']
dz_1, data_z1 = PDFs['dz_1, PDF_z1']
dz_2, data_z2 = PDFs['dz_2, PDF_z2']

dts = PDFs["dts"]

In [ ]:
dts

In [ ]:
## Temps court dt1
PDF_x0 = np.mean(data_0, axis=0)
Gauss_x0 = gauss(dxCOURT, 0, np.sqrt(2*mean_Dx*dts[0])) 

PDF_z0 = np.mean(data_z0, axis=0)
Gauss_z0 = gauss(dzCOURT, 0, np.sqrt(2*mean_Dz*dts[0])) 


## Temps long dt3
PDF_x2 = np.mean(data_2, axis=0)
Gauss_x2 = gauss(dxLONG, 0, np.sqrt(2*mean_Dx*dts[2]))

PDF_z2 = np.mean(data_z2, axis=0)
Gauss_z2 = gauss(dzLONG, 0, np.sqrt(2*mean_Dz*dts[2]))

In [ ]:
## Pour des temps de 0.01, 1.09 et 95.4 secondes

# PDFs =  loadmat("SIMU-PDFs123.mat", squeeze_me=True)

# indice_tau = PDFs["indice des \tau_1_2_3"]
# dx_1, data_1 = PDFs["x, PDF1"]
# dx_0, data_0 = PDFs["x, PDF2"]
# dx_2, data_2 = PDFs["x, PDF3"]

# Dxx = PDFs["Dpara_mean"]

# A4, D4, C4 = PDFs["coef:A4,D4,C4"]

In [ ]:
times = np.linspace(dt,Nt*dt, Nt)

dts = times[indice_tau]
dts

In [ ]:
Convergence_court = A4*tau_x[n_court]**2
Convergence_mid = 24*D4*tau_x[n_mid]-24*C4 #C4_x[n_mid]
Convergence_long = 24*D4*tau_x[n_long]-24*C4

In [ ]:
dXXlong =  np.linspace(-60, +60, 1000)
PDF_theory_x_long = P_Di_short_time(dXXlong*1e-6, dts[2], D_para, a, eta, B, lD, lB, H)

In [ ]:
# PDF_theory_x_long
dts[2]

In [ ]:
Gauss_x2 = gauss(dXXlong*1e-6, 0, np.sqrt(2*mean_Dx*dts[2]))

In [ ]:
fig = plt.figure(figsize = (3.3, 5/1.68),  tight_layout=True)
taille_marque = 3

## PDF1
ax2 = plt.subplot(2,2,1)
ax2.semilogy(dx_0*1e6, PDF_x0/np.trapz(PDF_x0, dx_0*1e6), "o", color="tab:blue", label=r"$X_t$", markersize=taille_marque)
ax2.semilogy(dXX, PDF_theory_x / np.trapz(PDF_theory_x, dXX), "k-", linewidth=1.5)
ax2.semilogy(dxCOURT*1e6, Gauss_x0/np.trapz(Gauss_x0, dxCOURT*1e6), "--", color="tab:cyan", linewidth=1.2)
ax2.set_xlim(-0.35, 0.35)
ax2.set_ylim(1e-10, 40)
ax2.set(
    # xlabel = r"$\Delta X_n$ ($\mu$m)",
    ylabel = r"$P(\Delta r_{i,n}, \tau_0)$  ($\mu$m$^{-1}$)",
)
ax2.text(0.08, 0.85, r"(a)", transform=ax2.transAxes)
locmaj = mpl.ticker.LogLocator(base=10.0, subs=(1.0, ), numticks=100)
locmin = mpl.ticker.LogLocator(base=10.0, subs=np.arange(2, 10) * .1, numticks=100)
ax2.yaxis.set_major_locator(locmaj)
ax2.yaxis.set_minor_locator(locmin)
ax2.yaxis.set_minor_formatter(mpl.ticker.NullFormatter())
ax2_ticks = ax2.get_yticklabels()
for i in ax2_ticks:
    i.set_visible(False)
for n, i in enumerate(ax2_ticks):
    if n%2==0:i.set_visible(True)

# Convergence 

In [1]:
dxLONG = np.linspace(-60e-6, 60e-6, 100)
dxMID = np.linspace(-15e-6, 15e-6, 100)
dxCOURT = np.linspace(-0.4e-6, 0.4e-6, 1000)

NameError: name 'np' is not defined

In [ ]:
Calcul du C4 en fonction de Ns

# Méthode <PDF> => C4
Nss= np.unique(np.logspace(0, 6, num = 40, base = 10, dtype=int))
C4_x_Ns_1 = np.zeros(len(Nss))
C4_x_Ns_0 = np.zeros(len(Nss))
C4_x_Ns_2 = np.zeros(len(Nss))

for n, i in enumerate(Nss):
        PDF = np.sum(data_1[:i, :], axis=0)
        PDF = PDF / np.trapz(PDF, dx_1)
        C4_x_Ns_1[n] = np.trapz(dx_1**4 *PDF, dx_1) - 3* (np.trapz(dx_1**2 * PDF, dx_1))**2
        
        PDF = np.sum(data_0[:i, :], axis=0)
        PDF = PDF / np.trapz(PDF, dx_0)
        C4_x_Ns_0[n] = np.trapz(dx_0**4 *PDF, dx_0) - 3* (np.trapz(dx_0**2 * PDF, dx_0))**2
        
        PDF = np.sum(data_2[:i, :], axis=0)
        PDF = PDF / np.trapz(PDF, dx_2)
        C4_x_Ns_2[n] = np.trapz(dx_2**4 *PDF, dx_2) - 3* (np.trapz(dx_2**2 * PDF, dx_2))**2
        
        


In [ ]:
# # MÉTHODE 2 : PDFs => C4s => <C4>
# C4_x_Ns_11 = np.zeros(len(Nss))
# C4_x_Ns_00 = np.zeros(len(Nss))
# C4_x_Ns_22 = np.zeros(len(Nss))

# import warnings
# warnings.filterwarnings("error")

# C4_1 = []
# C4_0 = []
# C4_2 = []
# for j in tqdm(range(len(data_1[:, 0]))):
#     if (sum(data_1[j, :]) != 0 and sum(data_0[j, :])!=0 and sum(data_2[j, :])!=0):
#         PDF = data_1[j, :] / np.trapz(data_1[j, :], dx_1)
#         C4_1.append(np.trapz(dx_1**4 *PDF, dx_1) - 3* (np.trapz(dx_1**2 * PDF, dx_1))**2)
        
#         PDF = data_0[j, :] / np.trapz(data_0[j, :], dx_0)
#         C4_0.append(np.trapz(dx_0**4 *PDF, dx_0) - 3* (np.trapz(dx_0**2 * PDF, dx_0))**2)
        
        
#         PDF = data_2[j, :] / np.trapz(data_2[j, :], dx_2)
#         C4_2.append(np.trapz(dx_2**4 *PDF, dx_2) - 3* (np.trapz(dx_2**2 * PDF, dx_2))**2)
        
# for n, i in enumerate(Nss):
#     C4_x_Ns_11[n] = np.mean(C4_1[:i])
#     C4_x_Ns_00[n] = np.mean(C4_0[:i])
#     C4_x_Ns_22[n] = np.mean(C4_2[:i])

# from scipy.io import savemat


    

In [ ]:
# Convergence_c4_3temps_M2 = {}
# Convergence_c4_3temps_M2["3 temps (s)"] = [Ntime[n_court]*dt, Ntime[n_mid]*dt, Ntime[n_long]*dt,]
# Convergence_c4_3temps_M2["C4_1"] = C4_x_Ns_00
# Convergence_c4_3temps_M2["C4_2"] = C4_x_Ns_11
# Convergence_c4_3temps_M2["C4_3"] = C4_x_Ns_22
# Convergence_c4_3temps_M2["Ns"] = Nss

# # savemat("Convergence_C4_methode2.mat", Convergence_c4_3temps_M2)

In [ ]:
Convergence_M2 = loadmat("Convergence_C4_methode2.mat", squeeze_me=True)
C4_x_Ns_11 = Convergence_M2['C4_2']
C4_x_Ns_00 = Convergence_M2['C4_1']
C4_x_Ns_22 = Convergence_M2['C4_3']
Nss = Convergence_M2['Ns']

# REPONSE REFERY PAPIER ALEXANDRE 2023

In [ ]:
Convergence_court,Convergence_mid,Convergence_long

In [ ]:
err_court1 = np.abs(Convergence_court-C4_x_Ns_1[-1])/Convergence_court * 100
err_court2 =  np.abs(Convergence_court-C4_x_Ns_11[-1])/Convergence_court * 100

err_mid1 = np.abs(Convergence_mid-C4_x_Ns_0[-1])/Convergence_mid * 100
err_mid2 = np.abs(Convergence_mid-C4_x_Ns_00[-1])/Convergence_mid * 100

err_long1 = np.abs(Convergence_long-C4_x_Ns_2[-1])/Convergence_long * 100
err_long2 = np.abs(Convergence_long-C4_x_Ns_22[-1])/Convergence_long * 100


In [ ]:
(err_court1, err_court2), (err_mid1, err_mid2), (err_long1, err_long2) 

In [ ]:
analytic = np.loadtxt('../C4_Datas/THEORY_EIGENVALUES_FOR_C4.txt',dtype="float" )

analytic_time = analytic[:,0]
analytic_C4 = analytic[:,1]
analytic_kurtosis = ((analytic_C4 + 3*(2*mean_Dx*analytic_time)**2) / (2*mean_Dx*analytic_time)**2) - 3

In [ ]:
theoriePDF0p01 = loadmat("../C4_Datas/PDF_ForExpsParam_and_t=0p01_zMax=4_Nmesh=400.mat")

theo_PDF_1 = theoriePDF0p01["P"]
theo_dx_1 = theoriePDF0p01["x"]

In [ ]:
theoriePDF1p09 = loadmat("../C4_Datas/PDF_ForExpsParam_and_t=1p09_zMax=4_Nmesh=400.mat")

theo_PDF_0 = theoriePDF1p09["P"]
theo_dx_0 = theoriePDF1p09["x"]

In [ ]:
theoriePDF95p4 = loadmat("../C4_Datas/PDF_ForExpsParam_and_t=95p4_zMax=8_Nmesh=400.mat")

theo_PDF_2 = theoriePDF95p4["P"]
theo_dx_2 = theoriePDF95p4["x"]

In [ ]:
## SEMI-EXPERIENCE NICO LE CHAUVAIN

experience = loadmat("../C4_Datas/C4_kurtosis_semi_experimental_exp_20211019_from_20221003.mat")

experience_C4 = experience["C4"][0]
experience_errC4 = experience["err_C4"][0]
experience_Kurtosis = experience["kurtosis"][0]
experience_errKurtosis = experience["err_kurtosis"][0]
experience_time = experience["time"][0]


experience_dX_0p01 = experience["pdf_0"]["bins_center"][0][0][0]
experience_PDF_0p01 = experience["pdf_0"]["pdf"][0][0][0]
experience_PDFstd_0p01 = experience["pdf_0"]["pdf_err"][0][0][0]

experience_dX_1 = experience["pdf_2"]["bins_center"][0][0][0]
experience_PDF_1 = experience["pdf_2"]["pdf"][0][0][0]
experience_PDFstd_1 = experience["pdf_2"]["pdf_err"][0][0][0]
# experience


In [ ]:
import matplotlib as mpl
from matplotlib.ticker import FormatStrFormatter

In [ ]:
# data_plot = {}
# data_plot["LagTime_simu_x"] = tau_x
# data_plot["C4_simu_x"] = C4_x
# data_plot["Kurtosis_simu_x"] = Kurtosis_x

# data_plot["LagTime_simu_z"] = tau_z
# data_plot["C4_simu_z"] = C4_z
# data_plot["Kurtosis_simu_z"] = Kurtosis_z

# data_plot["indice des \tau_1_2_3"] = np.array([n_court, n_mid, n_long])
# data_plot["COEF_C4_x : A4,D4,C4"] = np.array([A4_theo_x, D4_long_theo, C4_long_theo])
# data_plot["COEF_C4_z : A4"] = np.array([A4_theo_z])
# data_plot["PARAMETRES:_a_eta_H_lD_lB_B_Ntsub"] = np.array([a, eta, H, lD, lB, B, Nt_sub]) 

# from scipy.io import savemat
# savemat("SIMU-C4-Kurtosis-x,z_Nt_1e6_dt_1e-2.mat", data_plot)
 

In [ ]:
fig = plt.figure(figsize = (2*3.375,10),  tight_layout=True)
gs = mpl.gridspec.GridSpec(6,6)
# C4
axC4 = plt.subplot(gs[0, :])

axC4.plot(np.linspace(5e-3, 100, 1000), A4_theo_x*np.linspace(5e-3, 100, 1000)**2, "-", color="peru", linewidth=1.5 )
axC4.plot(np.linspace(30, 3000, 1000), (24*D4_long_theo*np.linspace(30, 3000, 1000)-24*C4_long_theo), "--", color="peru", linewidth=1.5)
axC4.loglog(analytic_time[:-25], analytic_C4[:-25], "k-", linewidth=1.5)
axC4.loglog(tau_x[:-5], C4_x[:-5], ".", color="tab:blue", markersize=8)
axC4.errorbar(experience_time[:-3], experience_C4[:-3], yerr=experience_errC4[:-3], fmt='*', linestyle="",color="tab:green", markersize=8)

axC4.annotate(r"$\tau _1$", (tau_x[n_court], C4_x[n_court]*2), 
             xytext=(tau_x[n_court], C4_x[n_court]*1000), ha="center",
            arrowprops=dict(arrowstyle = "->", lw = 0.8, color= "k"), fontsize=10)
axC4.annotate(r"$\tau _2$", (tau_x[n_mid], C4_x[n_mid]/2), 
             xytext=(tau_x[n_mid], C4_x[n_mid]/5000), ha="center",
            arrowprops=dict(arrowstyle = "->", lw = 0.8, color= "k"), fontsize=10)
axC4.annotate(r"$\tau _3$", (tau_x[n_long], C4_x[n_long]/2),
             xytext=(tau_x[n_long], C4_x[n_long]/5000), ha="center",
            arrowprops=dict(arrowstyle = "->", lw = 0.8, color= "k"), fontsize=10)
axC4.set(
    xlabel = r"$\tau$ (s)",
    ylabel = r"$\langle X_t^4 \rangle_\mathrm{c}$ ($\mathrm{m}^{4}$)",
    xlim = [5e-3, 1e3],
)
axC4.text(0.02, 0.9, r"(a)", transform=axC4.transAxes)   

locmaj = mpl.ticker.LogLocator(base=10.0, subs=(1.0, ), numticks=100)
axC4.xaxis.set_major_locator(locmaj)
locmin = mpl.ticker.LogLocator(base=10.0, subs=np.arange(2, 10) * .1, numticks=100)
axC4.xaxis.set_minor_locator(locmin)
axC4.xaxis.set_minor_formatter(mpl.ticker.NullFormatter())

locmaj = mpl.ticker.LogLocator(base=10.0, subs=(1.0, ), numticks=100)
axC4.yaxis.set_major_locator(locmaj)
locmin = mpl.ticker.LogLocator(base=10.0, subs=np.arange(2, 10) * .1, numticks=100)
axC4.yaxis.set_minor_locator(locmin)
axC4.yaxis.set_minor_formatter(mpl.ticker.NullFormatter())


# KURTOSIS
axKU = plt.subplot(gs[1, :])
axKU.plot(np.linspace(5e-3, 100, 1000), np.ones(1000)*A4_theo_x/(4*mean_Dx**2), "-", color="peru", linewidth=1.5)
axKU.plot(np.linspace(1, 3000, 1000), 24*D4_long_theo/(4*mean_Dx**2) /np.linspace(1, 3000, 1000) , "--", color="peru", linewidth=1.5)
axKU.loglog(analytic_time[:-25], analytic_kurtosis[:-25], "k-", linewidth=1.5)
axKU.loglog(tau_x[:-5], Kurtosis_x[:-5], ".", color="tab:blue", markersize=8)
axKU.errorbar(experience_time[:-3], experience_Kurtosis[:-3], yerr=experience_errKurtosis[:-3], fmt='*', linestyle="",color="tab:green", markersize=8)
axKU.annotate(r"$\tau _1$", (tau_x[n_court], Kurtosis_x[n_court]/1.1), 
             xytext=(tau_x[n_court], Kurtosis_x[n_court]/8), ha="center",
            arrowprops=dict(arrowstyle = "->", lw = 0.8, color= "k"), fontsize=10)
axKU.annotate(r"$\tau _2$", (tau_x[n_mid], Kurtosis_x[n_mid]/1.08), 
             xytext=(tau_x[n_mid], Kurtosis_x[n_mid]/8), ha="center",
            arrowprops=dict(arrowstyle = "->", lw = 0.8, color= "k"), fontsize=10)
axKU.annotate(r"$\tau _3$", (tau_x[n_long], Kurtosis_x[n_long]/1.1),
             xytext=(tau_x[n_long], Kurtosis_x[n_long]/8), ha="center",
            arrowprops=dict(arrowstyle = "->", lw = 0.8, color= "k"), fontsize=10)
axKU.set(
    xlabel = r"$\tau$ (s)",
    ylabel = r"$\frac{\langle X_t^4 \rangle }{ \langle X_t^2 \rangle^2} - 3$",
)
axKU.set_xlim([5e-3, 1e3])
axKU.set_ylim([7e-4, 2e-1])
# axKU.text(6.5e-3, 1.2e-1, r"(b)", transform=axKU.transData)
axKU.text(0.02, 0.9, r"(b)", transform=axKU.transAxes)   

locmaj = mpl.ticker.LogLocator(base=10.0, subs=(1.0, ), numticks=100)
axKU.xaxis.set_major_locator(locmaj)
locmin = mpl.ticker.LogLocator(base=10.0, subs=np.arange(2, 10) * .1, numticks=100)
axKU.xaxis.set_minor_locator(locmin)
axKU.xaxis.set_minor_formatter(mpl.ticker.NullFormatter())

locmaj = mpl.ticker.LogLocator(base=10.0, subs=(1.0, ), numticks=100)
axKU.yaxis.set_major_locator(locmaj)
locmin = mpl.ticker.LogLocator(base=10.0, subs=np.arange(2, 10) * .1, numticks=100)
axKU.yaxis.set_minor_locator(locmin)
axKU.yaxis.set_minor_formatter(mpl.ticker.NullFormatter())



# PDF1
ax1 = plt.subplot(gs[2, 0:2])
Z1 = np.trapz(data_1, dx_1*1e6)
# ax1.semilogy(dxCOURT*1e6, Gauss1_x/ np.trapz(Gauss1_x, dxCOURT*1e6) , "--", color="darkgrey", linewidth=1.5)
ax1.semilogy(theo_dx_1, theo_PDF_1, "k-", linewidth=1.5)
ax1.semilogy(-theo_dx_1, theo_PDF_1, "k-", linewidth=1.5)
ax1.errorbar(experience_dX_0p01*1e6, experience_PDF_0p01/ np.trapz(experience_PDF_0p01, experience_dX_0p01*1e6), yerr=experience_PDFstd_0p01/ np.trapz(experience_PDF_0p01, experience_dX_0p01*1e6), fmt='*', linestyle="",color="tab:green", markersize=8)
ax1.semilogy(dx_1*1e6, data_1/ Z1,".", color="tab:blue", markersize=8)
# ax1.fill_between(dx_1*1e6, Min / Z1, Max / Z1, color='blue', alpha=0.1) #ZONE BLEU MIN/MAX DES Ns TRAJECTOIRES
# ax1.text(0.14, 14, r"$\tau_1$", fontsize=6)
ax1.set_xlim(-0.35, 0.35)
ax1.set_ylim(1e-10, 5e1)
ax1.set(
    xlabel = r"$x$ ($\mu$m)",
    ylabel = r"$p( x, \tau_1)$ ($\mu$m$^{-1}$)",
)
ax1.text(0.07, 0.9, r"(c)", transform=ax1.transAxes)   

locmaj = mpl.ticker.LogLocator(base=10.0, subs=(1.0, ), numticks=100)
ax1.yaxis.set_major_locator(locmaj)
locmin = mpl.ticker.LogLocator(base=10.0, subs=np.arange(2, 10) * .1,
                                      numticks=100)
ax1.yaxis.set_minor_locator(locmin)
ax1.yaxis.set_minor_formatter(mpl.ticker.NullFormatter())




#PDF2
ax0 = plt.subplot(gs[2, 2:4])
Z0 = np.trapz(data_0, dx_0*1e6)
# ax0.plot(dxMID*1e6, Gauss0_x / np.trapz(Gauss0_x, dxMID*1e6), "--", color="darkgrey", linewidth=1.5)
ax0.semilogy(theo_dx_0, theo_PDF_0, "k-", linewidth=1.5)
ax0.semilogy(-theo_dx_0, theo_PDF_0, "k-", linewidth=1.5)
ax0.semilogy(dx_0*1e6, data_0/ Z0,".", color="tab:blue", markersize=8)
norm=np.trapz(experience_PDF_1, experience_dX_1*1e6)
ax0.errorbar(experience_dX_1*1e6, experience_PDF_1/norm, yerr=experience_PDFstd_1/norm, fmt='*', linestyle="",color="tab:green", markersize=8)
# ax0.fill_between(dx_0*1e6, Min / Z0, Max / Z0, color='blue', alpha=0.1)

ax0.set_xlim(-3.5, 3.5)
ax0.set_ylim(1e-10, 5e0)
#ax0.text(1.5, 4e-1, r"$\tau_2 = $ "+ str(Ntime[n_mid]*dt) + " s", fontsize=6)
# ax0.text(1.5, 4e-1, r"$\tau_2$", fontsize=6)

ax0.set(
    xlabel = r"$x$ ($\mu$m)",
    ylabel = r"$p( x, \tau_2)$ ($\mu$m$^{-1}$)",
)
ax0.text(0.07, 0.9, r"(d)", transform=ax0.transAxes)   

locmaj = mpl.ticker.LogLocator(base=10.0, subs=(1.0, ), numticks=100)
ax0.yaxis.set_major_locator(locmaj)
locmin = mpl.ticker.LogLocator(base=10.0, subs=np.arange(2, 10) * .1,
                                      numticks=100)
ax0.yaxis.set_minor_locator(locmin)
ax0.yaxis.set_minor_formatter(mpl.ticker.NullFormatter())



#PDF3
ax2 = plt.subplot(gs[2, 4:6])
Z2 = np.trapz(data_2, dx_2*1e6)
ax2.semilogy(theo_dx_2, theo_PDF_2, "k-", linewidth=1.5)
ax2.semilogy(-theo_dx_2, theo_PDF_2, "k-", linewidth=1.5)
# ax2.semilogy(dxLONG*1e6, Gauss2_x / np.trapz(Gauss2_x, dxLONG*1e6), "--", color="darkgrey", linewidth=1.5)
ax2.plot(dx_2*1e6, data_2 / Z2, ".", color="tab:blue", markersize=8)
# ax2.fill_between(dx_2*1e6, Min / Z2, Max / Z2, color='blue', alpha=0.1)

ax2.set_xlim(-30, 30)
ax2.set_ylim(1e-10, 5e-1)
# ax2.text(13, 3e-2, r"$\tau_3 = $ "+ str(Ntime[n_long]*dt) + " s", fontsize=8)
# ax2.text(13, 3e-2, r"$\tau_3 $", fontsize=8)
ax2.set(
    xlabel = r"$ x$ ($\mu$m)",
    ylabel = r"$p( x, \tau_3)$ ($\mu$m$^{-1}$)",
)
#ax2.text(-28, 3e-2, r"(e)", transform=ax2.transData)  
ax2.text(0.07, 0.9, r"(e)", transform=ax2.transAxes)   
locmaj = mpl.ticker.LogLocator(base=10.0, subs=(1.0, ), numticks=100)
ax2.yaxis.set_major_locator(locmaj)
locmin = mpl.ticker.LogLocator(base=10.0, subs=np.arange(2, 10) * .1,
                                      numticks=100)
ax2.yaxis.set_minor_locator(locmin)
ax2.yaxis.set_minor_formatter(mpl.ticker.NullFormatter())



ax0_ticks = ax0.get_yticklabels()
for i in ax0_ticks:
    i.set_visible(False)
    
for n, i in enumerate(ax0_ticks):
    if n%2==0:i.set_visible(True)
    

ax1_ticks = ax1.get_yticklabels()
for i in ax1_ticks:
    i.set_visible(False)
    
for n, i in enumerate(ax1_ticks):
    if n%2==0:i.set_visible(True)


###    
ax2_ticks = ax2.get_yticklabels()
for i in ax2_ticks:
    i.set_visible(False)
    
for n, i in enumerate(ax2_ticks):
    if n%2==0:i.set_visible(True)
    

    
# ###
# axC4_ticks = axC4.get_xticklabels()
# for i in axC4_ticks:
#     i.set_visible(False)
    
# for n, i in enumerate(axC4_ticks):
#     if n%2==0:i.set_visible(True)
# ###

# axKU_ticks = axKU.get_xticklabels()
# for i in axKU_ticks:
#     i.set_visible(False)
    
# for n, i in enumerate(axKU_ticks):
#     if n%2==0:i.set_visible(True)

ax0.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax1.xaxis.set_major_formatter(FormatStrFormatter('%g'))


fig.tight_layout(pad = 0.001, h_pad=0.1)

# plt.savefig("../Figures/Simu_C4-Kurtosis-PDF_Large-Gap_H-40micron_V2.pdf",bbox_inches='tight')

In [ ]:
# Calcul du C4 en fonction de Ns

# Méthode <PDF> => C4
Nss= np.unique(np.logspace(0, 6, num = 40, base = 10, dtype=int))
dC4x_t = np.zeros(len(tau_x))

for m, j in enumerate(tau_x):
    for n, i in enumerate(Nss):
            dx, pdf = PDFs["dx_0, PDF_x0"]
            PDF = np.sum(data_0[:i, :], axis=0)
            PDF = PDF / np.trapz(PDF, dx_0)
            C4_x_Ns_M1_0[n] = np.trapz(dx_0**4 *PDF, dx_0) - 3* (np.trapz(dx_0**2 * PDF, dx_0))**2

        